In [1]:
!python --version


Python 3.6.10 :: Anaconda, Inc.


In [2]:
import os
os.getcwd()

'C:\\Users\\User-Pc\\BerTopic'

In [10]:
!pip install -U sentence-transformers==2.1.0
!pip install konlpy
!pip install umap
!pip install -U scikit-learn

  Using cached umap-0.1.1.tar.gz (3.2 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=7dc7340c118d1dda8e10b6cec9b40a58a8cac15335f0eb6c20c1f3de5a0819c8
  Stored in directory: c:\users\user-pc\appdata\local\pip\cache\wheels\df\22\c9\3de2d976b162d3f90c60ed7e415f344d0630518a501093e7b1
Successfully built umap
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=ebd533245f5b4d050a21e818e28ef03cfb43d07ed8362ab9a26e4ba755afaf95
  Stored in directory: c:\users\user-pc\appdata\local\pip\cache\wheels\23\9d\42\5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [3]:
import pandas as pd
path = "./data/"
file_list = os.listdir(path)
dfs = []
for i in file_list:
    df = pd.read_excel(path+i, engine = "openpyxl")
    dfs.append(df)
    print(i)


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20180101-20181231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20190101-20191231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20200101-20201231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20210101-20211231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20220101-20220810.xlsx


In [65]:
data = pd.concat(dfs)

In [66]:
from sentence_transformers import SentenceTransformer
from datetime import datetime

import logging
import re

import numpy as np
import pandas as pd

In [6]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [7]:
model_path = "./kpfSBERT"
model = SentenceTransformer(model_path) 

In [8]:
# 데이터 전처리
data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
cluster_mode = '특성추출(가중치순 상위 50개)'
corpus = data[cluster_mode].values.tolist()

for i, cp in enumerate(corpus):
    corpus[i] = cp.split(',')

In [209]:
df = data[['일자']]
df['키워드'] = corpus
df['일자'] = [str(x)[0:4] for x in df['일자']] 

<ipython-input-209-f8a2c0bae374>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['키워드'] = corpus
<ipython-input-209-f8a2c0bae374>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일자'] = [str(x)[0:4] for x in df['일자']]


In [210]:
keyword_vocab = []
for i in df['키워드']:
    keyword_vocab.extend(i)
keyword_vocab = list(set(keyword_vocab))

In [211]:
from collections import Counter

In [212]:
years = ['2018','2019','2020','2021','2022']

tmp_df_list = []
for year in years:
    print(year)
    tmp = []
    for i in df[df['일자']== year]['키워드']:
        tmp.extend(i)
    tmp_df = pd.DataFrame(pd.Series(Counter(tmp)))
    tmp_df = tmp_df.reset_index()
    tmp_df = tmp_df.rename(columns = {0:'cnt','index':'keyword'})
    tmp_df_list.append(tmp_df)

2018
2019
2020
2021
2022


In [213]:
tmp_df_list[0]

,keyword,cnt
0,지점장,313
1,영업부장,59
2,영업부,69
3,센터장,351
4,조사역,20
...,...,...
36191,강명규,1
36192,이진석,1
36193,배현재,1
36194,노진용,1


In [214]:
df = tmp_df_list[0].merge(tmp_df_list[1], on = 'keyword', how = 'outer')

In [215]:
for i in range(len(tmp_df_list)):
    if i > 1:
        df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')

<ipython-input-215-efd5dda4b60b>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')


In [216]:
col = ['keyword'] + years

In [217]:
df.columns = col

In [218]:
df

,keyword,2018,2019,2020,2021,2022
0,지점장,313.0,347.0,362.0,273.0,168.0
1,영업부장,59.0,37.0,24.0,22.0,4.0
2,영업부,69.0,154.0,114.0,142.0,112.0
3,센터장,351.0,386.0,442.0,463.0,217.0
4,조사역,20.0,30.0,40.0,9.0,13.0
...,...,...,...,...,...,...
126147,일본증시,NaN,NaN,NaN,NaN,1.0
126148,호랑이띠,NaN,NaN,NaN,NaN,1.0
126149,닛케이225지수,NaN,NaN,NaN,NaN,1.0
126150,항공우주국,NaN,NaN,NaN,NaN,1.0


In [219]:
# 2022년 수치 날짜 보정

In [220]:
df['2022'] = round(df['2022']*(365/223)) 

In [221]:
df

,keyword,2018,2019,2020,2021,2022
0,지점장,313.0,347.0,362.0,273.0,275.0
1,영업부장,59.0,37.0,24.0,22.0,7.0
2,영업부,69.0,154.0,114.0,142.0,183.0
3,센터장,351.0,386.0,442.0,463.0,355.0
4,조사역,20.0,30.0,40.0,9.0,21.0
...,...,...,...,...,...,...
126147,일본증시,NaN,NaN,NaN,NaN,2.0
126148,호랑이띠,NaN,NaN,NaN,NaN,2.0
126149,닛케이225지수,NaN,NaN,NaN,NaN,2.0
126150,항공우주국,NaN,NaN,NaN,NaN,2.0


In [222]:
df.index = df['keyword']

In [223]:
df = df.drop('keyword', axis = 1)

In [224]:
df['tot_cnt'] = df.sum(axis = 1)

In [225]:
df = df.T

In [226]:
df['year_cnt'] = df.sum(axis = 1)

In [227]:
df = df.T

In [259]:
df

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
지점장,313.0,347.0,362.0,273.0,275.0,1570.0
영업부장,59.0,37.0,24.0,22.0,7.0,149.0
영업부,69.0,154.0,114.0,142.0,183.0,662.0
센터장,351.0,386.0,442.0,463.0,355.0,1997.0
조사역,20.0,30.0,40.0,9.0,21.0,120.0
...,...,...,...,...,...,...
호랑이띠,0.0,0.0,0.0,0.0,2.0,2.0
닛케이225지수,0.0,0.0,0.0,0.0,2.0,2.0
항공우주국,0.0,0.0,0.0,0.0,2.0,2.0


In [261]:
# 연도별 건수 보정
df['2018'] = df['2018'][:-1]*10000 / df['2018'][-1]
for year in years:
    df[year] = df[year][:-1]*10000 / df[year][-1]

In [265]:

df

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
지점장,16.782932,13.786199,11.768378,7.633054,7.822723,1570.0
영업부장,3.163556,1.469998,0.780224,0.615118,0.199124,149.0
영업부,3.699752,6.118371,3.706064,3.970307,5.205666,662.0
센터장,18.820476,15.335656,14.369124,12.945436,10.098424,1997.0
조사역,1.072392,1.191890,1.300373,0.251639,0.597372,120.0
...,...,...,...,...,...,...
호랑이띠,0.000000,0.000000,0.000000,0.000000,0.056893,2.0
닛케이225지수,0.000000,0.000000,0.000000,0.000000,0.056893,2.0
항공우주국,0.000000,0.000000,0.000000,0.000000,0.056893,2.0


In [229]:
df =df.fillna(0)

In [241]:
df[df.index == '농협은행']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
농협은행,301.0,427.0,531.0,540.0,340.0,2139.0


In [242]:
df[df.index == '국민은행']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
국민은행,500.0,697.0,505.0,567.0,435.0,2704.0


In [250]:
df[df.index == '하나은행']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
하나은행,357.0,519.0,374.0,624.0,640.0,2514.0


In [243]:
df[df.index == '신한은행']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
신한은행,703.0,605.0,621.0,979.0,908.0,3816.0


In [247]:
df[df.index == '카카오뱅크']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
카카오뱅크,118.0,174.0,206.0,424.0,210.0,1132.0


In [249]:
df[df.index == '우리은행']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
우리은행,352.0,567.0,518.0,662.0,509.0,2608.0


In [248]:
df[df.index == '토스뱅크']

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
토스뱅크,0.0,59.0,17.0,81.0,77.0,234.0


In [267]:
from scipy import stats

x = [1,2,3,4,5]

slopes = []
for i in list(df.values):
    y = i[:-1]
    slope, intercept, r, p, std_err = stats.linregress(x, y)
    slopes.append(slope)

In [266]:
df2 = df.copy()
df2['slope'] = slopes

In [237]:
df2.sort_values(by = 'slope',ascending = True)[0:50]

,2018,2019,2020,2021,2022,tot_cnt,slope
keyword,,,,,,,
블록체인,1101.0,995.0,642.0,764.0,777.0,4279.0,-87.9
사진제공,121.0,227.0,101.0,4.0,5.0,458.0,-45.5
keb하나은행,138.0,168.0,6.0,0.0,0.0,312.0,-44.4
keb,134.0,150.0,8.0,0.0,0.0,292.0,-41.8
최종구,65.0,255.0,7.0,1.0,0.0,328.0,-38.4
위성호,176.0,28.0,0.0,1.0,0.0,205.0,-37.9
베트남,302.0,459.0,198.0,181.0,259.0,1399.0,-36.4
이대훈,82.0,184.0,46.0,0.0,0.0,312.0,-34.8
빅데이터,917.0,1072.0,1402.0,1119.0,720.0,5230.0,-34.7


In [236]:
df2.sort_values(by = 'slope',ascending = False)[0:50]

,2018,2019,2020,2021,2022,tot_cnt,slope
keyword,,,,,,,
year_cnt,186499.0,251701.0,307604.0,357655.0,351540.0,1454999.0,43603.6
코로나19,0.0,0.0,4017.0,2846.0,1735.0,8598.0,631.6
미국,1092.0,1292.0,1595.0,2441.0,2188.0,8608.0,334.1
온라인,565.0,720.0,1915.0,2097.0,1512.0,6809.0,327.1
투자자,472.0,574.0,758.0,1433.0,1509.0,4746.0,293.3
메타버스,0.0,1.0,0.0,755.0,1082.0,1838.0,291.8
가상자산,3.0,25.0,199.0,673.0,1120.0,2020.0,288.2
마이데이터,36.0,104.0,557.0,966.0,974.0,2637.0,273.8
빅테크,2.0,36.0,583.0,952.0,853.0,2426.0,261.8


In [203]:
# BERT 임베딩
corpus_embeddings = model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/2153 [00:00<?, ?it/s]

In [302]:
one

,2018
지점장,313
영업부장,59
영업부,69
센터장,351
조사역,20
...,...
강명규,1
이진석,1
배현재,1
노진용,1


In [171]:
len(corpus_embeddings)

68885

In [172]:
clustering = DBSCAN(eps=0.4, min_samples=10,metric = 'cosine').fit(corpus_embeddings)

In [173]:
set(clustering.labels_)

{-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [175]:
df['Topic'] = clustering.labels_

In [184]:
df[df['Topic'] == 7]

,뉴스 식별자,일자,제목,Topic
13181,1.100701e+06,20200403,“관객 90%가 사라졌다” 영화업계 “심장 멈춰 당장 심폐소생해야”,7
13188,2.100101e+06,20200403,"영화계 코로나19 기근에 영화인연대 측 “정부 지원, 골든타임 놓친다”",7
13200,1.101101e+06,20200402,영화계 “영화관련업 특별지원업종 즉각 지정하라”,7
13210,2.100101e+06,20200402,"영화계 “코로나19로 영화산업 붕괴 초읽기, 정부 실질적인 지원 필요”(전문)",7
13219,8.100401e+06,20200402,"코로나19영화인연대 측 ""정부 대책, 뜬구름...특별지원업종 지정 원해""",7
13556,8.100401e+06,20200325,"""영화산업 붕괴 위기, 정책 실행할 때""...韓영화계, 정부 지원 촉구 (종합)",7
13561,2.100801e+06,20200325,"붕괴 직면한 영화계, 수수방관 문체부 영진위 비판",7
13567,1.100201e+06,20200325,"“코로나19→산업 붕괴 대량 실업” 영화계, 긴급지원 요청",7
13570,1.100801e+06,20200325,"코로나로 붕괴 위기 영화계, ""정부 지원 절실하다""",7
13571,2.100701e+06,20200325,“영화산업도 고사위기” 정부 지원 ‘SOS’,7


In [136]:
df[df['Topic'] == 11]

,뉴스 식별자,일자,제목,Topic
3546,2.100101e+06,20191002,"LG이노텍, 협력사대표 임원 초청 `2019 동반성장 아카데미` 개최",11
3567,2.100201e+06,20191002,"""협력사에 실질적 지원"" LG이노텍, '동반성장 아카데미' 개최",11
3571,7.101201e+06,20191002,"LG이노텍, 협력사 상생 위한 `동반성장 아카데미` 개최",11
8657,2.100851e+06,20200720,"​정철동 LG이노텍 대표 ""사회적 책임 성실 이행 사랑받는 기업 되겠다""",11
8265,1.100701e+06,20220225,"LG이노텍, 올해부터 협력사에 ESG 컨설팅 지원하기로",11
8275,2.100601e+06,20220224,"정철동 LG이노텍 사장, 협력사에 ESG 컨설팅",11
8296,2.100101e+06,20220224,"LG이노텍, 협력사 ESG 컨설팅 나선다",11
8299,7.101201e+06,20220224,"LG이노텍, 협력사와 동반성장 강화 ""실질적인 상생활동 실행""",11
8304,2.100311e+06,20220224,"LG이노텍, ‘2022 동반성장 상생데이’ 개최",11
8305,2.100801e+06,20220224,"LG이노텍, 협력사와 동반성장 상생 100여개 업체와 협약 체결",11


In [142]:
df[df['Topic'] == 17]

,뉴스 식별자,일자,제목,Topic
10113,1.100401e+06,20190307,VR체험 트럭-폐차 견적비교 앱 나온다,17
10116,7.101201e+06,20190307,이동형 VR체험 폐차 견적비교 규제 샌드박스 실증 특례 대상,17
10120,1.100201e+06,20190307,VR 체험 트럭 폐차 중개앱 나온다,17
10124,2.100311e+06,20190306,‘VR체험 트럭’ ‘폐차 견적앱’ 나온다 ICT 규제 샌드박스 통과,17
10127,2.100601e+06,20190306,찾아가는 'VR 체험 트럭' 나온다,17
10128,2.100101e+06,20190306,폐차 중개앱 VR 체험트럭 나온다,17
10129,1.100901e+06,20190306,VR 트럭 모바일 폐차 앱 나온다 블록체인 송금 서비스는 '불투명',17
10138,2.100201e+06,20190306,'VR 체험 트럭' '폐차 중개 앱' 나온다,17
10143,2.100851e+06,20190306,"""VR체험 트럭으로 폐차 중개 온라인으로 가능해진다""",17
10146,1.101001e+06,20190306,트럭 이용한 가상현실 체험 사업화 가능해졌다,17


In [169]:
df[df['Topic'] == -1]

,뉴스 식별자,일자,제목,Topic
21,2.100311e+06,20181231,"최종구-윤석헌, 신년사도 '온도차'",-1
26,2.100851e+06,20181231,"[신년사]김상조 공정거래위원장(전문)...""공정경제 구현, 경기 나쁘다고 후퇴하는 ...",-1
35,2.100601e+06,20181231,최고점 대비 85%↓ 2018년 달군 '가상화폐 10대 뉴스',-1
39,1.100301e+06,20181231,밀레니얼 세대는 '가치 투자' 선호,-1
44,2.100101e+06,20181231,[2018 Hot-Line] `고의 분식` 삼성바이오로직스 투자자 냉온탕 오가,-1
...,...,...,...,...
11612,2.100701e+06,20220101,"2021년 수출 수입액 사상 최대 12월 무역수지, 20개월만에 적자(종합)",-1
11614,2.100801e+06,20220101,"[2022 신년사] 문인 광주 북구청장 ""완전한 회복, 미래 대전환에 전력 쏟을 것""",-1
11615,2.100601e+06,20220101,"日증시, 지난해 5% 올랐지만 호랑이해는 '불안해' [정영효의 일본산업 분석]",-1
11617,2.100311e+06,20220101,[신년 기고]과학기술이 대한민국의 미래다,-1


In [61]:
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [62]:
tf_idf, count = c_tf_idf(corpus, m=len(corpus))

<ipython-input-61-0f534e09bd9e>:7: RuntimeWarning: invalid value encountered in true_divide
  tf = np.divide(t.T, w)


In [66]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, corpus, n=20)


AttributeError: 'list' object has no attribute 'Topic'

In [ ]:
topic_sizes = extract_topic_sizes(rslt[category_id]); topic_sizes.head(10)

In [60]:
''' JAVA HOME 문제 해결 필요
from konlpy.tag import Kkma
 
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result
    
custom_tokenizer = CustomTokenizer(Kkma())
'''

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
 model.get_topic_freq().head()

In [ ]:
 model.get_topic(49)

In [ ]:
model.visualize_topics(

In [ ]:
model.visualize_distribution(probabilities[0])